# 3D CNN

In [17]:
import tensorflow as tf

import nibabel as nib
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy import ndimage
from pathlib import Path

import sys
sys.path.append(r"/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification")

import brain_mets_classification.custom_funcs as funcs

from tqdm import tqdm

## Prepare data for training
To-do:
- load nifti files
- rotate images by 90 degrees
- rescale the values to be between -1 and 1
- compress all 4 sequences into one array (while using the different sequences as different "colors")
- add data augmentation
- one hot encode sex
- preload images
- separate images into train and test set (maybe 90:10)

CAVE: remove patient sub-01383503 from training

In [28]:
path_to_preprocessed_images: Path = Path("/Volumes/BrainMets/Rgb_Brain_Mets/brain_mets_classification/derivatives/preprocessed_20240131-135755")

def load_patient(patientID):
    """loads the images for a specific patient"""
    images = []
    # get all four sequences
    patient_path = Path(patientID)
    image_names = os.listdir(path_to_preprocessed_images / patient_path)
    
    # load them
    for name in image_names:
        path_to_image = path_to_preprocessed_images / patient_path / Path(name)
        image = nib.load(path_to_image)
        data = image.get_fdata()
        tensor = tf.convert_to_tensor(data, dtype = float)
        images.append(data)
    
    if len(images) != 4:
        print(f"Warning: either too many or too few images for {patientID} (#{len(images)})")
    
    # return four images as array
    return images

def rotate_90_deg(images):
    """rotates images by 90 degrees"""
    # rotate images
    rotated_images = []
    for image in images:
        rotated_images.append(ndimage.rotate(image, angle = 90))

    # return back
    return rotated_images

@tf.py_function(Tout=tf.float32)
def pad_images(images, target_shape = (155,185,149)): # for further information why this specific values is used please take a look at dataset_analysis.ipynb
    """adds \"zero\" padding to the images, the value of a corner of the image gets used to padding"""
    padded_images = []

    # get value to use for padding
    for image in images:
        # gets value of an image corner
        min_value = image[:,:,0][0][0]

        # Calculate the padding amounts for each dimension
        current_shape = image.shape
        pad_widths = []

        for target_dim, current_dim in zip(target_shape, current_shape):
            total_padding = max(0, target_dim - current_dim)
            padding_before = total_padding // 2
            padding_after = total_padding - padding_before
            pad_widths.append((padding_before, padding_after))
        
        # pad the image
        padded_image = tf.pad(tensor = image,
               paddings = pad_widths,
               mode = "CONSTANT",
               constant_values = int(min_value))
        
        padded_images.append(padded_image)

    # return images
    return padded_images

@tf.py_function(Tout=tf.float32)
def rescale_images(images):
    """rescales the values for each image pixel (or voxel) to be between -1 and 1"""
    # rescale images
    # return images
    pass

@tf.py_function(Tout=tf.float32)
def merge_images(images):
    """merge images so that the fourth dimension used for the different sequences"""
    # merge image
    # return array
    pass



In [29]:
patient_images = rotate_90_deg(pad_images(load_patient("sub-02063373")))

plt.imshow(patient_images[0][:,:,90], cmap="inferno")

print(patient_images[0].shape)

2024-02-09 12:04:26.218129: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: ValueError: Tensor conversion requested dtype float32 for Tensor with dtype float64: <tf.Tensor: shape=(155, 185, 149), dtype=float64, numpy=
array([[[-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        ...,
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.]],

       [[-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        ...,
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.]],

       [[-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        ...,
        [-3., -3., -3., ..., -3., -3., -3.],
  

InvalidArgumentError: {{function_node __wrapped__EagerPyFunc_Tin_1_Tout_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} ValueError: Tensor conversion requested dtype float32 for Tensor with dtype float64: <tf.Tensor: shape=(155, 185, 149), dtype=float64, numpy=
array([[[-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        ...,
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.]],

       [[-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        ...,
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.]],

       [[-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        [-3., -3., -3., ..., -3., -3., -3.],
        ...,
        [-3.

## Create AI architecture
To-do:
- custom loss (dice loss) or mutli-class loss https://medium.com/@zergtant/use-weighted-loss-function-to-solve-imbalanced-data-classification-problems-749237f38b75